# 从0到1训练大模型
# Part 1 数据收集与数据预处理

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/11.jpg)

## 0 开篇：MateConv训练之旅

在人工智能领域，大模型的崛起正掀起一股新的浪潮。随着技术的不断突破，大模型已逐渐成为AI发展不可或缺的重要基石。无论是在自然语言处理、计算机视觉，还是其他领域，大模型展现了前所未有的潜力。而随着计算能力的提升与行业需求的细化，**越来越多的岗位需要针对特定应用场景开发专业的大模型，比如编程助手、医疗诊断工具、法律专业大模型，甚至是用于保密环境的专属模型**。

然而，尽管大模型已经如此热门，互联网上关于如何从零开始训练一套大模型的详细资料却凤毛麟角、各岗位的面试问题围绕原理深度展开、训练经验可谓十分宝贵。这种空白促使我们踏上了探索之路、亲自开启训练，我们希望通过亲身实践，**为大模型课程学员们贡献第一手的经验与资料、同时为你们提供一个非常强有力的实践案例**。

我们训练的大模型名为 **MateConv**，这一旅程从初始的“小试牛刀”到目标明确的1B规模模型，凝聚了许多的努力。正如老师们在答疑群中提到，考虑到训练投入的成本、这门课程不仅知识很“值钱”、物理上也很“值钱”。以下是我们的训练概况：

| 模型名称     | 模型参数量级      | 数据量级                | 硬件资源               | 训练时间              | 训练成本                |
| ------------ | ----------------- | ----------------------- | ---------------------- | --------------------- | ----------------------- |
| **MateConv mini** | 0.02B<br>（两千万参数） | 约60G（分词器约1G、预训练约35G、微调约20G） | RTX 3090 x2           | 全流程约3-4天          | AutoDL租赁，约250元     |
| **MateConv**      | 1B<br>（10亿参数）      | 约3~4T文本+1T代码（实际收集超过10T数据、如果你们有更大的运存可以使用更大的数据集） | A800 x 8，硬盘4T      | 全流程约1个多月（预训练约20天） | AutoDL租赁2.5\~3万元；购置设备需80\~100万元，电费约3000r |

为了实现MateConv的训练，我们参考了国内外大量的语言模型训练代码和资料，结合团队的摸索与实验，逐步搭建起了这一套完整的训练流程。尽管由于资源限制、以及团队还在同步进行其他大模型产品开发、我们未能穷尽所有可能的训练手段和理论，但我们已投入百万级的训练与人力成本，将这份经验倾囊相授，只为更多有志于大模型训练的团队和个人能够有更好的学习基础与资料、同时为大家抛砖引玉。

接下来的时间里，我们会将这次从0到1的训练历程、技术细节、优化经验与心得完整记录在课程中。欢迎你和我们一起训练、共同探索不同设备、不同数据下的更多可能性，我们期待与更多的大模型技术人一起，推动大模型技术的发展，为行业带来更多可能！

MateConv项目的第一步已经迈出，一起来吧！

In [ ]:
如何让传统文本转变成大模型能够识别的数据？

文字/段落  ==> chunkize ==> tokenizer（token）==> encoder[0,1,2,3,4]  ==> 词向量构成

- **大模型训练流程概览与讲解方式**

从 **0到1** 训练大模型是一个复杂而系统的工程，需要涵盖从数据准备到模型部署的多个环节。以下是一个完整的流程框架：

| 流程            | 说明                                                                                           |
|:---------------:|--|
| **数据准备**        | 收集高质量、覆盖面广的训练数据，对其进行清洗、去噪和格式化处理。划分<br>训练集、验证集，并存储为高效读取的格式。这一步为模型提供了扎实的输入基础。 |
| **硬件与环境配置**  | 为模型训练准备高性能硬件（如 A800、A100 GPU），搭建分布式训练环境，<br>并优化深度学习框架的配置。这一步确保训练效率和稳定性。 |
| **分词器训练**      | 根据训练数据量和模型任务需求，选择适合的分词算法（如 BPE 或 <br>SentencePiece）。分词器决定了模型如何理解数据，是数据与模型的桥梁。 |
| **设计模型架构**    | 选择适合的模型结构（如 GPT、BERT），并配置参数量、层数、激活函数<br>等细节。对于大规模任务，可以结合领域特点定制模型。 |
| **预训练**        | 使用无监督任务从海量数据中提取通用知识，比如语言模型的自回归建模或<br>掩码建模。预训练的效果直接影响模型后续的微调能力。 |
| **意图对齐微调**        | 通过监督微调（SFT）或强化学习对齐（RLHF），让模型学习人类偏好，<br>避免输出无意义或有害内容。对齐步骤是模型实用化的关键。 |
| **特定优化微调**  | 在特定任务（如文本分类、问答）上微调模型，结合冻结与解冻层的策略进一步<br>优化性能，满足应用需求。                          |
| **模型量化**       | 通过剪枝、量化和知识蒸馏等技术优化模型，提高推理效率，降低计算与存储<br>成本，使模型更适合部署环境。                            |
| **部署与监控**      | 将模型部署到生产环境中，使用推理优化工具提升服务效率，同时通过实时<br>监控与用户反馈不断改进模型性能和可靠性。                     |

**在课程中我们呈现的环节有——**

| 流程            | MateConv Mini<br> (0.02B) | MateConv <br> (1B) | 单独的案例 |
|:---------------:|:---------------------:|:-------------:|:-:|
| 数据准备        | ✅ | ✅ | ✅ |
| 硬件与环境配置  | ✅                     | ✅             | ✅|
| 分词器训练      | ✅  | --              |--|
| 设计模型架构    | ✅  | ✅     |✅|
| 预训练          | ✅                     | ✅             |--|
| 意图对齐微调        | ✅                     | ✅             |--|
| 特定优化微调  | --                     | --             |✅|
| 模型量化       | --                   | --             |✅|
| 部署与监控      | ✅                     | --             |--|

在进行课程讲解时，每个环节我们将采用【技术概览 - MateConv Mini代码实现 - MateConv代码实现 - 深度原理或补充】的4段式结构进行讲解、你可以挑选自己最感兴趣的部分进行学习。

**在开始正式课程之前、千万需要先阅览硬件配置指南以及Autodl环境配置流程**。确保你已经搭建好线上环境、并且在finalshell中执行了下面这些流程 ↓

```shell

#建立线上虚拟环境，命名为MateConv
conda create --name MateConv python=3.10
conda init
source ~/.bashrc
conda activate MateConv

#创建Jupyter Kernel
conda install jupyterlab
conda install ipykernel
python -m ipykernel install --user --name MateConv --display-name "Python (MateConv)"

#创建项目主目录
cd ~/autodl-tmp
mkdir MateConv

#打开jupyter
cd ~/autodl-tmp/MateConv
jupyter lab --allow-root

#根据requirements.txt配置环境
touch requirements.txt
cd ~/autodl-tmp/MateConv
pip install -r requirements.txt -i https://pypi.tuna.tsinghua.edu.cn/simple
```

## 1 MateConv Mini的数据收集与数据预处理

数据是大模型训练的核心，质量和数量直接影响模型效果。在我们进行数据准备时，我们要考虑到模型的具体用途、模型的规模以及模型的精度要求等等、这些都会影响我们对数据的收集和参考。在大模型进行训练时，我们的数据来源主要有两种——一种是公开的、已经收集整理好的数据集、主要来自于Huggingface上已开源的各种中英文/代码数据集，另一种则是存储于数据库或者干脆就是以文件形式存储的原始数据（raw data）、核心来源是我们收集的1000份PDF行业报告。

### 1.1 MateConv Mini所使用的数据集

MateConv Mini所使用的语料都是开源数据集、以中文为主、并且以轻量、高质量、少流程为主要追求、希望能够在最小的数据量级内获得最好的成果。分词器数据虽小，但能满足小模型实验需求；预训练数据覆盖广泛且经过清洗和去重，确保了多样性和高质量，对小规模模型的泛化能力有良好支持；意图对齐数据集经过精细设计，能帮助模型对齐人类意图，提升微调阶段的表现。然而，分词器数据词表覆盖范围有限，可能在特定领域的适应性上存在不足，同时预训练数据和微调数据在深度领域信息上略显欠缺。整体而言，这些数据集为 MateConv Mini 提供了扎实的训练基础，但在扩展到更大模型或特定领域时，还需补充规模更大、领域更专的高质量数据。

| 流程       | 数据集名称 | 数据量级 | 数据概况 |
|:---------------:|:---------------------:|:-------------:|:-:|
| Tokenizer训练流程 | Huggingface开源<br>Tokenizer_train.jsonl | 1G | **jsonl超短文本数据集**<br><br>开源的超小型词表数据集，总词表量只有6400，几乎是所有开源Tokenizer数据集中最小的|
| 预训练流程   | huggingface开源<br>序列猴子通用文本数据集 | 31G | **jsonl短文本数据集**<br><br>序列猴子通用文本数据集由来自**网页、百科、博客、问答、开源代码、书籍、报刊、专利、教材、考题等多种公开可获取的数据进行汇总清洗之后而形成的大语言模型预训练语料**。它将不同来源的HTML、TEXT、PDF、EPUB等各类格式的数据统一整理为JSONL格式，并进行了仔细的筛选、去重、清洗和**价值对齐**，从而形成了一份覆盖全面、规模庞大、安全可信、质量上乘的预训练语料。|
| 意图对齐微调   | ModelScope开源<br>匠数科技大模型SFT数据集 | 18G  | **jsonl对话（问答对数据）**<br><br>匠数大模型SFT数据集是从**网络上的公开数据源收集并整理得来**，经过细致的数据清洗、格式统一，最终获得了用于大模型SFT的包含10M条数据的中文数据集和包含2M条数据的英文数据集。|

我们为你准备了这些数据集，你都可以在课程所对应的网盘中找到他们 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/12.png)

<font color = "red">**导入数据查看一下**，</font>这一步你可以在本地jupyter上操作。

In [6]:
import json

# 文件路径
file_paths = [
    r"D:\pythonwork\2025LLMtraining\MateConvMini\MateConv_Dataset\sft_data_zh.jsonl",
    r"D:\pythonwork\2025LLMtraining\MateConvMini\MateConv_Dataset\tokenizer_train.jsonl",
    r"D:\pythonwork\2025LLMtraining\MateConvMini\MateConv_Dataset\mobvoi_seq_monkey_general_open_corpus.jsonl",
]

# 读取并展示文件内容
def read_and_display_samples(file_path, num_samples=2):
    print(f"Reading from: {file_path}")
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i >= num_samples:
                    break
                data = json.loads(line.strip())
                print(f"Sample {i+1}: {json.dumps(data, indent=4, ensure_ascii=False)}")
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
    print("\n ============================= \n")

# 遍历文件并读取样本
for path in file_paths:
    read_and_display_samples(path)

Reading from: D:\pythonwork\2025LLMtraining\MateConvMini\MateConv_Dataset\sft_data_zh.jsonl
Sample 1: {
    "id": 1,
    "instruction": "",
    "input": "好的。现在请你将这个文本中的所有的逗号都替换成空格。",
    "output": "好的，请稍等一下，现在我会将文本中的所有逗号替换为空格。处理后文本为：\"这是一个句子 目的是看看是否可以正确地从这个句子中删除关键词。\"。处理结果如何？",
    "history": [
        [
            "给定一段文本和关键词列表，删除文本中包含所有给定关键词的子字符串。\n文本：\"这是一个测试句子，目的是看看模型是否可以正确地从这个句子中删除关键词。\"\\n关键词列表：[‘测试’，‘模型’]",
            "删除包含所有给定关键词的子字符串后，文本变为：\"这是一个句子，目的是看看是否可以正确地从这个句子中删除关键词。\""
        ]
    ],
    "language": "chinese",
    "data_source": "https://huggingface.co/datasets/BelleGroup/train_3.5M_CN",
    "input_len": 59,
    "output_len": 66,
    "num_utter": 2,
    "type": 31,
    "type_keyword": [
        "字符串",
        "代码",
        "函数",
        "编写",
        "实现",
        "给定",
        "使用",
        "输入",
        "文本",
        "程序"
    ]
}
Sample 2: {
    "id": 2,
    "instruction": "",
    "input": "帮我回答一道历史题目。清朝时期的八旗共有多少旗人？",
    "output": "清朝时期八旗旗人总数约为200万人左右，其中正黄旗、正蓝旗、正

<font color ="red">**相信你已经注意到了，在不同的训练流程中、我们需要不同类型的数据——**

**1. 微调数据集：问答对**
- **如何实现预训练**：
  - **任务类型**：监督微调（SFT, Supervised Fine-Tuning）。
  - **训练方法**：基于人类标注的问答对，通过优化模型在给定输入（问题）下生成期望输出（回答）的能力。
    - 输入：问题文本（或带上下文的对话历史）。
    - 输出：目标回答文本。
  - **损失函数**：通常使用**交叉熵损失（Cross-Entropy Loss）**，评估模型生成回答与参考答案的相似程度。交叉熵损失会在每个时间步计算预测的分布与目标token的真实分布之间的差异，对于一段话、交叉熵损失通过将每个token的损失相加、得到整段话的损失。

**2. 预训练数据集：短文本**
- **如何实现预训练**：
  - **任务类型**：无监督学习（例如自回归语言建模或掩码语言建模）。
    - **自回归建模**（如 GPT）：模型按顺序预测每个词的下一个词。
    - **掩码语言建模**（如 BERT）：模型根据上下文预测被掩盖的词。
  - **训练方法**：利用短文本数据片段，构建语言建模任务。
    - 自回归任务：输入前半部分文本，预测后续部分。此时使用的是**自回归损失CLM**。
    - 掩码任务：随机掩盖部分单词，让模型填空。此时使用的是**掩码损失MLM**。
    - 多模态任务：通常对图文进行匹配。此时使用的是**对比损失**。

**3. 分词器训练数据集：超短文本**
- **如何实现预训练**：
  - **任务类型**：词汇构建与分词规则学习。
  - **训练方法**：
    1. 对超短文本进行统计分析，确定出现频率最高的字符或词。
    2. 使用分词算法（如 BPE 或 SentencePiece）将常见的字符组合压缩为词表项。
    3. 生成分词规则和词表，供后续模型训练使用。
    4. 由于分词器的训练本质上是一个基于规则或统计算法的过程，因此不是神经网络的优化过程，因此分词器的训练过程**并不需要损失函数**。

<font color ="red">**除此之外，我们还需要认识一下Jsonl数据格式**

JSONL（JSON Lines）：JSONL（JSON Lines）是一种特别适合处理大规模数据的格式，尤其在机器学习和大数据领域得到了广泛应用，它是一种逐行存储 JSON 对象的文件格式，每行是一个独立的 JSON 对象，行与行之间并没有特定的结构。每行的 JSON 对象独立存在，不属于同一个数组或对象。例如：

In [10]:
{"name": "Alice", "age": 25}
{"name": "Bob", "age": 30}

{'name': 'Bob', 'age': 30}

这种数据类型展示的信息其实就是——

|name|age|
|:-:|:-:|
|Alice|25|
|Bob|30|

但由于JSONL天生的字典格式、它展示表单信息的效率远远高于Dataframe这些结构，因此许多大型数据都呈现jsonl格式，你可能看到超大数据集是这样的结构 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/13.png)

但是，JSONL有它的劣势，其中最核心的一条就是文件体积稍大、它还不是最有效存储超大数据的格式——

| 格式          | 优势                                                                 | 劣势                                                          |
|---------------|----------------------------------------------------------------------|---------------------------------------------------------------|
| **JSONL**     | 支持流式处理、高容错性、易扩展、通用性强                              | 文件体积稍大（每条记录有元数据开销）。                         |
| **CSV**       | 紧凑、高效，占用空间小                                               | 不适合嵌套或复杂结构数据，不支持多种数据类型。                 |
| **Parquet**   | 高压缩比、支持列式存储、查询速度快                                    | 不易阅读，不适合直接调试或小规模任务。                         |
| **Protobuf**  | 高效、压缩率高、适合大规模数据传输                                   | 二进制文件不可读，需要额外的工具和定义文件解析。               |
| **二进制文件** | 存储高效，占用空间小，适合高密度数据（如矩阵或图像）                  | 不可读、不易调试，对文件结构高度依赖。                         |

针对这样的数据集，我们可以完成一个极为特殊的预处理手段，那就是将JSONL格式转化为二进制Bin文件，不过这个操作需要在我们完成Tokenzier训练后才可以实现。

### 2.1 MateConv Mini的tokenizer训练

**下面这段代码你需要在服务器的jupyter中运行，请在你的服务器中上传好tokenizer_jsonl数据集。**

In [ ]:
huggingface  ==> library (transformers、diffuser、Dataset)

- step 0. 服务器配置

```shell
#激活已建好的虚拟环境
conda activate MateConv

#进入MateConv文件夹
cd ~/autodl-tmp/MateConv/

#在MateConv文件夹中建立dataset目录
mkdir dataset

#进入dataset目录
cd ./dataset
```

接着，找到网盘中的数据文件Tokenizer_jsonl，上传到dataset文件夹内 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/12.png)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/88.png)

上传完毕后即可使用！整个分词器的训练流程（下面的训练代码）也已打包好一个单独的ipy文件（Tokenizer_Training.ipy），可在网盘中下载。你可以将这段代码同样上传至线上的`~/autodl-tmp/MateConv/`文件夹、然后在这个文件夹下启动jupyter，即可运行这段代码。

```shell

cd ~/autodl-tmp/MateConv/

jupyter lab --allow-root
```

启动jupyter之后，需要使用autodl的ssh隧道工具进行代理、才可以访问线上jupyter，相信你在看这节课之前已经看过环境配置与jupyter访问指南。打开代理页面之后就可以在左侧目录中直接看到tokenizer_training.ipy文件了。接下来就可以在线上运行这段文件、来实现Tokenizer的训练了。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/89.png)

- Step 1.导入必要的库

In [2]:
import random
from tqdm import tqdm
from transformers import AutoTokenizer
import json
from datasets import load_dataset
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
import os

- Step 2.读取 tokenizer_train.jsonl 文件

In [3]:
def read_texts_from_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            yield data['text']

# 测试读取数据，你可以自定义你的目录
data_path = '/root/autodl-tmp/MateConv/dataset/tokenizer_train.jsonl'
texts = read_texts_from_jsonl(data_path)

# 打印前几行文本
for i, text in enumerate(texts):
    if i < 5:
        print(text)
    else:
        break

好的。现在请你将这个文本中的所有的逗号都替换成空格。 好的，请稍等一下，现在我会将文本中的所有逗号替换为空格。处理后文本为："这是一个句子 目的是看看是否可以正确地从这个句子中删除关键词。"。处理结果如何？
帮我回答一道历史题目。清朝时期的八旗共有多少旗人？ 清朝时期八旗旗人总数约为200万人左右，其中正黄旗、正蓝旗、正白旗、正红旗的人数较多，其他旗的人数较少。
嗯，谢谢你介绍的做法很详细，但我不喜欢吃鸡蛋，有没有其他菜做法能介绍一下？ 当然，你可以试试酸辣土豆丝这道菜。
材料：
土豆2个、红椒1个、青椒1个、大葱1根、醋、生抽、盐、鸡精、料酒
做法：
1.土豆去皮，切成丝；红椒和青椒切成细丝；大葱切段备用。
2.热锅凉油，油热后放入土豆丝，煸炒至变软。
3.倒入红椒、青椒和大葱段，继续煸炒至熟。
4.加入适量的盐、鸡精、料酒和生抽，翻炒均匀。
5.最后，加入适量的醋，翻炒均匀即可。
小贴士：
1. 土豆切丝时，可以放入淡盐水中泡一下，这样可以去除多余的淀粉。
2. 煮土豆丝时，不要煮得太久，以免烂糊。
3. 加入醋的时候，根据自己的口味多少来进行调节，一般来说，盐与醋的比例为1:1。
4. 如果喜欢辣味可以加入一些干辣椒丝。
希望你会喜欢这道酸辣土豆丝！
请描述一下如何正确规划个人理财。 正确规划个人理财需要以下几个步骤：
1.了解自己的财务状况。这包括收入、支出、资产和负债等信息。了解自己的财务状况可以帮助人们更好地制定财务计划。
2.设定财务目标。需要考虑短期目标和长期目标，例如以年为单位设定的支出计划、购房、购车等的长期目标。
3.制定预算计划。在了解自己的财务状况并设定财务目标后，需要制定一个预算计划。这可以帮助人们控制支出、节省开支并达到财务目标。
4.理性投资和储蓄。人们可以投资于股票、基金、房产或其他投资渠道以实现财务目标。但在投资前需了解相关知识并进行风险评估。同时还应储蓄一定金额，以应对突发事件或为达成某些目标做准备。
5.审时度势，合理调整。财务计划需要不断地审时度势，根据实际情况做出调整，以达到最终的财务目标。需要注意财务状况的变化、投资的收益和风险等因素。
通过以上五个步骤，人们可以做到合理规划个人理财，掌握自己的财务命运，更好地实现自己的财务目标。
描述一下天堂和地狱的生态系统和环境。 天堂和地狱被认为是灵性信仰中关于死后世界的两种不同概

- Step 3.初始化分词器

首先，通过 `models.BPE()` 创建了一个基于 Byte-Pair Encoding (BPE) 模型的分词器。BPE 是一种常用于文本分词的子词分解算法，特别在自然语言处理任务中被广泛使用，如机器翻译和语言模型训练。BPE 的主要思想是通过将频繁出现的字符或字符对合并成一个新的子词单元，逐步构建一个子词级别的词汇表，从而处理词汇表稀疏性和未登录词问题。

In [5]:
# 初始化tokenizer
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

# 定义特殊token
special_tokens = ["<unk>", "<s>", "</s>"]

# 设置训练器并添加特殊token
trainer = trainers.BpeTrainer(
    vocab_size=6400,
    special_tokens=special_tokens,  # 确保这三个token被包含
    show_progress=True,
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet()
)

print("分词器初始化成功，准备训练。")

分词器初始化成功，准备训练。


- Step 4.训练分词器

【TIME Warning：10mins】

In [6]:
# 读取文本数据
texts = read_texts_from_jsonl(data_path)

# 训练tokenizer
tokenizer.train_from_iterator(texts, trainer=trainer)

print("分词器训练完成！")




分词器训练完成！


- Step 5.保存分词器

在训练完毕之后，还需要设置解码器 (`tokenizer.decoder = decoders.ByteLevel()`) ，这是为了在生成文本时正确地将分词器产生的 token 序列还原回原始文本。

同时，在保存tokenizer之前，你需要建立用于存放模型的目录 ↓

```shell
#激活已建好的虚拟环境
conda activate MateConv

#进入MateConv文件夹
cd ~/autodl-tmp/MateConv/

#在MateConv文件夹中建立dataset目录
mkdir -p ~/autodl-tmp/MateConv/model/mateconv_tokenizer

#进入dataset目录
cd ./model/mateconv_tokenizer
```

In [7]:
# 设置解码器
tokenizer.decoder = decoders.ByteLevel()

# 保存tokenizer
tokenizer_dir = "/root/autodl-tmp/MateConv/model/mateconv_tokenizer"
os.makedirs(tokenizer_dir, exist_ok=True)
tokenizer.save(os.path.join(tokenizer_dir, "tokenizer.json"))
tokenizer.model.save("/root/autodl-tmp/MateConv/model/mateconv_tokenizer")

# 手动创建配置文件
config = {
    "add_bos_token": False,
    "add_eos_token": False,
    "add_prefix_space": True,
    "added_tokens_decoder": {
        "0": {
            "content": "<unk>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
            },
        "1": {
            "content": "<s>",
         -p ~/autodl-tmp/MateConv/model/mateconv_tokenizer   "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
            },
        "2": {
            "content": "</s>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
            }
    },
    "bos_token": "<s>",
    "clean_up_tokenization_spaces": False,
    "eos_token": "</s>",
    "legacy": True,
    "model_max_length": 1000000000000000019884624838656,
    "pad_token": None,
    "sp_model_kwargs": {},
    "spaces_between_special_tokens": False,
    "tokenizer_class": "PreTrainedTokenizerFast",
    "unk_token": "<unk>",
    "use_default_system_prompt": False,
    "chat_template": "{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ system_message }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<s>user\\n' + content + '</s>\\n<s>assistant\\n' }}{% elif message['role'] == 'assistant' %}{{ content + '</s>' + '\\n' }}{% endif %}{% endfor %}"
}

# 保存配置文件
with open(os.path.join(tokenizer_dir, "tokenizer_config.json"), "w", encoding="utf-8") as config_file:
    json.dump(config, config_file, ensure_ascii=False, indent=4)

print("Tokenizer 保存成功！")

Tokenizer 保存成功！


<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20241022194241795.png" alt="image-20241022194241795" style="zoom:33%;" />

- Step 6.评估分词器

In [2]:
from transformers import AutoTokenizer

# 加载预训练的tokenizer
tokenizer = AutoTokenizer.from_pretrained("./model/mateconv_tokenizer")

# 测试一段对话
messages = [
    {"role": "system", "content": "你是一个优秀的聊天机器人，总是给我正确的回应！"},
    {"role": "user", "content": '是椭圆形的'},
    {"role": "assistant", "content": '456'},
    {"role": "user", "content": '456'},
    {"role": "assistant", "content": '789'}
]

# 使用模板进行文本处理
new_prompt = tokenizer.apply_chat_template(messages, tokenize=True)
print(new_prompt)

[608, 1589, 4835, 269, 4833, 954, 4725, 270, 1170, 345, 4584, 5204, 1273, 648, 2207, 1, 320, 275, 201, 345, 1390, 258, 3852, 1081, 269, 2, 201, 1, 1078, 538, 501, 201, 22, 23, 24, 2, 201, 1, 320, 275, 201, 22, 23, 24, 2, 201, 1, 1078, 538, 501, 201, 25, 26, 27, 2, 201]


### 3.1 MateConv Mini的数据预处理流程

数据集需要手动下载**mobvoi_seq_monkey_general_open_corpus.zip**文件并上传至服务器上的文件夹内。**由于数据有13.5G，上传大约需要半小时时间**。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/12.png)

在这里你可以自己选择上传至服务器的目录，我选择的目录是 ↓ 

<center><img src="https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/90.png" alt="image-20241022200836959" style="zoom:100%;" />

- 文件解压缩

&emsp;&emsp;接下来则需要在服务器上运行如下命令：

```bash
conda activate MateConv

sudo apt-get install unzip

mkdir -p ~/autodl-tmp/MateConv/Data/seq-monkey #换成你自己的目录

cd /root/autodl-tmp/MateConv/Data/seq-monkey #进入你自己设置的目录、开始解压缩

unzip mobvoi_seq_monkey_general_open_corpus.zip #大约需要5~10mins时间，直到你看到jsonl的大小为32G为止
```

解压缩后就会得到文件`mobvoi_seq_monkey_general_open_corpus.jsonl`，文件的标准大小应该接近32G。刷新至文件大小不再增加、且命令行回复初始状态后，表示解压已结束。

<center><img src="https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/91.png" alt="image-20241022201037481" style="zoom:100%;" />

接下来，你则需要对数据进行处理，并将数据转换成二进制文件。下面这段代码已经被提取到ipy文件中（**Prepare_Train_Data.ipy**），同样你可以将其上传到MateConv默认命令中，并在代理服务器中运行它。

- Step 1.导入库

In [1]:
import itertools
import re
import json
import jsonlines
import psutil
import ujson
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from datasets import load_dataset
import os
from tqdm import tqdm

/root/miniconda3/envs/MateConv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- Step 2.定义BOS和EOS标记，并加载分词器

In [2]:
# 定义BOS和EOS标记
bos_token = "<s>"
eos_token = "</s>"

In [3]:
# 加载训练好的分词器路径
tokenizer = AutoTokenizer.from_pretrained('/root/autodl-tmp/MateConv/model/mateconv_tokenizer', use_fast=False)
print(f'加载的tokenizer词表大小: {len(tokenizer)}')

加载的tokenizer词表大小: 6400


- Step 3.读取部分数据

In [6]:
def preview_dataset(file_path, num_lines=5):
    """
    读取并展示数据集的前 num_lines 行
    """
    # 检查文件是否存在
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 文件不存在，请检查路径！")

    # 逐行读取并展示前 num_lines 行
    with jsonlines.open(file_path) as reader:
        for idx, obj in enumerate(reader):
            print(f"第 {idx + 1} 行数据: {obj}")
            if idx + 1 >= num_lines:
                break

# 指定文件路径和需要展示的行数
file_path = './dataset/mobvoi_seq_monkey_general_open_corpus.jsonl'
preview_dataset(file_path, num_lines=5)

第 1 行数据: {'text': '在查处虚开增值税专用发票案件中，常常涉及进项留抵税额和税款损失的认定和处理。在计算税款损失时，要不要将进项留抵税额包括在内？\n对此，实务中存在意见分歧。\n有人主张归并，即计算税款损失时包括进项留抵税额；\n有人主张剥离，即计算税款损失时剔除进项留抵税额。分析这个问题，需要确定进项留抵税额与税款损失之间是什么关系。\n理清这二者之间的关系，首先需要了解增值税的概念和其抵扣机制。增值税是以商品（货物、服务等）在流转过程中产生的增值额作为计税依据而征收的一种流转税。为避免重复征税，在增值税中存在抵扣链条机制。\n一般而言，交易上游企业缴纳的税额，交易下游企业可以对相应的税额进行抵扣。\n对增值税一般纳税人来说，其购进货物、服务等取得增值税专用发票，发票上的税额是进项税额。\n其出售货物、服务等，向购买方开具增值税专用发票，发票的税额是销项税额。\n一般情况下，销项税额减去进项税额的金额是应纳税额，企业根据应纳税额按期申报纳税。\n其次需要了解进项留抵税额的概念及产生原因。\n在计算销项税额和进项税额的差额时，有时会出现负数，即当期进项税额大于当期销项税额。这个差额在当期未实现抵扣，为进项留抵税额，在以后纳税人有销项税额时再进行抵扣。\n企业产生进项留抵税额的主要原因是其进项税额和销项税额时间上的不一致。\n例如，企业前期集中采购货物和服务，投资大，销项税率低于进项税率等。\n从税款抵扣的角度看，进项留抵税额只是购进的这部分进项税额参与到增值税应纳税额的计算过程中，但是其对应的进项税额抵扣还未真正实现，一般要等到其未来有相应的销项税额时，才能真正实现进项税额抵扣。\n可见，进项留抵税额处于不确定状态，能否抵扣受到很多因素影响，例如企业经营中断，没有销项税额，这时进项留抵税额就无法实现抵扣。但如果企业按照税收政策规定申请进项留抵退税，进项税额抵扣就随之实现。\n最后需要了解税款损失的概念。\n税款损失，通常是指因虚开增值税专用发票，导致国家税款被骗或者流失的金额。关于税款损失，实务中有多种表述。\n例如，北京大学法学院教授陈兴良曾谈到虚开行为本身不会造成国家税款损失，只有利用发票抵扣时才会造成国家税款损失。刘兵等编著的《虚开增值税专用发票案例司法观点和案例解析》一书中提到：“给国家税款造成损失的数额，实际上就是被骗取的国家税款在侦查终

> - **什么样的 JSONL 中会携带无效的 JSON 格式**？

在 JSONL 文件中，每一行都应当是有效的 JSON 对象，但有以下几种常见情况会导致无效的 JSON 行：

1. **缺少必要的符号**：JSON 对象必须用 `{}` 括起来，且键值对之间要用逗号分隔。例如，缺少结束花括号：
   ```
   {"name": "Alice", "age": 25
   ```

2. **引号问题**：JSON 的键和值（非数字类型）必须使用双引号。如果使用了单引号或忘记引号，都会导致无效格式：
   ```
   {'name': 'Alice', 'age': 25}  # 错误的引号
   ```

3. **数据未完整写入**：例如由于文件写入中断，某一行可能是不完整的 JSON 对象：
   ```
   {"name": "Alice", "age":
   ```

4. **额外的标点符号或换行符**：一些 JSONL 文件可能错误地加入了多余的标点符号或不正确的换行符，导致解析错误：
   ```
   {"name": "Alice", "age": 25},
   {"name": "Bob", "age": 30}
   ```

   这里的逗号在 JSONL 中是不需要的，因为每一行都是独立的。

5. **字符编码问题**：文件可能存在编码不一致的情况，特别是非 UTF-8 编码时，可能会引发解码错误，例如你的错误提示中看到的 "codec can't decode byte"。

> - **无效的 JSON 格式示例**

例如，以下 JSON 数据格式就是无效的：

- 缺少关闭括号：
   ```json
   {"name": "Alice", "age": 25
   ```

- 键名未加双引号：
   ```json
   {name: "Alice", "age": 25}
   ```

- 值部分使用了错误的引号：
   ```json
   {"name": 'Alice', "age": 25}
   ```

- Step 4.统计与清理数据

In [8]:
def get_total_lines(file_path):
    """
    获取 JSONL 文件的总行数，不忽略错误，保证能够全面统计。
    """
    with open(file_path, 'rb') as f:  # 使用二进制模式避免编码问题
        return sum(1 for _ in f)

In [12]:
def check_jsonl_format(file_path):
    """
    检查 JSONL 文件中的每一行是否是有效的 JSON 格式，带进度显示，并统计所有有问题的行。
    """
    total_lines = get_total_lines(file_path)  # 获取文件总行数
    valid_lines = 0
    invalid_lines = 0

    # 使用逐行读取，捕获 JSON 和编码错误
    with open(file_path, 'rb') as f:  # 使用二进制读取避免编码问题
        # 使用 tqdm 进度条显示检查进度
        for idx, line in tqdm(enumerate(f), total=total_lines, desc="Checking JSONL format"):
            try:
                # 先尝试将每行数据解码为 UTF-8
                decoded_line = line.decode('utf-8')
                # 然后检查是否是有效的 JSON 格式
                obj = jsonlines.Reader([decoded_line]).read()
                valid_lines += 1
            except UnicodeDecodeError as e:
                print(f"Encoding error at line {idx + 1}: {e}")
                invalid_lines += 1
            except jsonlines.InvalidLineError as e:
                print(f"Invalid JSON at line {idx + 1}: {e}")
                invalid_lines += 1

    print(f"检查完成，文件中共有 {valid_lines} 行有效的 JSON 数据，{invalid_lines} 行无效的 JSON 数据。")
    return valid_lines, invalid_lines

In [13]:
valid_lines, invalid_lines = check_jsonl_format(file_path)

Checking JSONL format: 100%|██████████| 9598787/9598787 [02:01<00:00, 79274.88it/s]

Encoding error at line 9598787: 'utf-8' codec can't decode byte 0xe5 in position 503: unexpected end of data
检查完成，文件中共有 9598786 行有效的 JSON 数据，1 行无效的 JSON 数据。


In [14]:
def remove_invalid_line(file_path, output_path, invalid_line_num):
    """
    读取文件，跳过指定的无效行，并将结果写入新文件
    """
    with open(file_path, 'rb') as infile, open(output_path, 'wb') as outfile:
        for idx, line in enumerate(infile):
            if idx + 1 != invalid_line_num:  # 跳过无效行
                outfile.write(line)

# 使用该函数删除第 9598787 行并保存为新文件
remove_invalid_line('./dataset/mobvoi_seq_monkey_general_open_corpus.jsonl',
                    './dataset/mobvoi_seq_monkey_general_open_corpus_cleaned.jsonl', 
                    invalid_line_num=9598787)

最终会得到如下的文件 ↓

<center><img src="https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/92.png" alt="image-20241022201401913" style="zoom:80%;" />

- Step 5.定义处理函数（逐块处理数据）

**注意修改函数中的目录为自己的目录！**

In [19]:
def process_seq_monkey(chunk_size=50000):
    """
    逐块读取 mobvoi_seq_monkey_general_open_corpus.jsonl 文件，
    对文本进行分词，并将分词结果保存为二进制文件，支持跳过无效行，并显示处理进度。
    """
    doc_ids = []
    chunk_idx = 0
    total_lines = 0

    # 先计算总行数以便显示进度
    # 注意调整成你自己的目录
    with open('./Data/seq-monkey/mobvoi_seq_monkey_general_open_corpus_cleaned.jsonl', 'r', encoding='utf-8') as f:
        total_lines = sum(1 for _ in f)

    # 打开jsonlines文件逐行读取
    with jsonlines.open('./Data/seq-monkey/mobvoi_seq_monkey_general_open_corpus_cleaned.jsonl') as reader:
        # 使用 tqdm 进度条显示进度
        with tqdm(total=total_lines, desc="Processing lines") as pbar:
            while True:
                try:
                    # 使用 itertools.islice 按块读取文件，每次读取 chunk_size 行数据
                    chunk = list(itertools.islice(reader, chunk_size))
                except jsonlines.InvalidLineError as e:
                    print(f"Skipping invalid chunk at chunk {chunk_idx}: {e}")
                    continue

                if not chunk:  # 如果读取到文件末尾，则停止
                    break

                # 遍历块中的每一行数据
                # 逐行对数据进行编码（按token进行编码）
                for idx, obj in enumerate(chunk):
                    try:
                        # 从每一行数据中提取'text'字段（即文本内容）
                        content = obj.get('text', '')
                        
                        # 跳过长度超过512的文本
                        if len(content) > 512:
                            continue

                        # 对文本进行分词，将其转为 token ids 序列，并加上BOS和EOS标记
                        text_id = tokenizer(f'{bos_token}{content}{eos_token}').data['input_ids']
                        
                        # 将分词结果添加到 doc_ids 列表中
                        doc_ids += text_id

                    except UnicodeDecodeError as e:
                        # 如果遇到编码错误，跳过该行，并打印错误信息
                        print(f"Skipping invalid line {chunk_idx * chunk_size + idx + 1}: {e}")
                        continue

                # 每处理完一块数据，更新 chunk_idx 并打印进度信息
                chunk_idx += 1
                pbar.update(len(chunk))  # 更新进度条

                # 如果累积的 token ids 超过 1,000,000 个，保存到文件中
                if len(doc_ids) > 1000000:
                    arr = np.array(doc_ids, dtype=np.uint16)
                    with open(f'./Data/seq-monkey/clean_seq_monkey.bin', 'ab') as f:
                        f.write(arr.tobytes())
                    doc_ids = []

    # 如果处理完所有数据后 doc_ids 中还有未保存的内容，最后再保存一次
    if doc_ids:
        arr = np.array(doc_ids, dtype=np.uint16)
        with open(f'./Data/seq-monkey/clean_seq_monkey.bin', 'ab') as f:
            f.write(arr.tobytes())

In [20]:
def pretrain_process():
    """
    函数的作用是调用 process_seq_monkey() 函数生成数据，
    然后整合所有生成的二进制文件，并将其合并保存为一个总的预训练数据文件。
    """
    # 调用 process_seq_monkey 函数处理数据
    process_seq_monkey()

    # 数据文件路径列表，目前只处理 clean_seq_monkey.bin 文件
    data_path_list = [
        './Data/seq-monkey/clean_seq_monkey.bin'
    ]
    
    data_lst = []
    
    # 读取生成的二进制文件
    for data_path in data_path_list:
        with open(data_path, 'rb') as f:
            # 将二进制文件中的内容加载到 numpy 数组中
            data = np.fromfile(f, dtype=np.uint16)
            data_lst.append(data)

    # 将所有读取到的数据合并为一个大数组
    arr = np.concatenate(data_lst)
    print(f"合并后的数据大小: {arr.shape}")

    # 将合并后的数据保存为最终的预训练数据文件
    with open('./Data/seq-monkey/pretrain_data.bin', 'wb') as f:
        f.write(arr.tobytes())

- **为什么要将数据转换为二进制文件**：

1. **高效存储和读取**：二进制文件相比文本文件具有更高的存储和读取效率，尤其是对于大规模的数据集。由于二进制文件是以原始的机器可读格式存储的，不需要进行字符编码转换，因此读取速度更快。对于预训练阶段通常需要处理大量数据，二进制文件可以显著减少读取时间。

2. **减少文件大小**：二进制格式的数据比常规的文本格式更紧凑，占用的磁盘空间更少。这对存储大数据集尤其重要，能够显著节省存储资源。

3. **与深度学习框架兼容**：深度学习框架（如 PyTorch、TensorFlow 等）在训练时往往需要数据以某种高效的格式加载到内存中。将数据保存为二进制格式有助于快速载入到 NumPy 数组或直接作为模型输入，避免了每次都需要重新转换。

4. **跨平台一致性**：二进制文件可以跨平台使用而不丢失精度和数据信息，适合在不同的硬件和操作系统环境中使用。

- 这个函数的具体作用：
> - `process_seq_monkey()` 函数负责处理原始数据并生成单个或多个二进制文件。
> - 然后这些生成的二进制文件通过 `np.fromfile` 读取并存入 NumPy 数组，所有的二进制数据都会被拼接到一起。
> - 最后，通过 `np.concatenate` 合并所有的 NumPy 数组，生成一个总的数据数组，并将其存储为一个大的二进制文件 (`pretrain_data.bin`)，供后续的模型预训练使用。

总结来说，这种处理方式主要是为了提高效率，方便在大规模预训练任务中快速加载数据并减少磁盘和内存的占用。

- 运行数据处理

In [22]:
pretrain_process()

Processing lines: 100%|██████████| 9598786/9598786 [36:17<00:00, 4408.66it/s]


合并后的数据大小: (1115541384,)


运行结束后会创建一个名为pretrain_data.bin的二进制数据文件，该文件也就是接下来进行模型预训练的文件：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20241023183247295.png" alt="image-20241023183247295" style="zoom:33%;" />

## 2. MateConv的数据收集与数据预处理

### 2.1 MateConv的预训练数据组成

MateConv所使用的语料是由开源数据集和我们自制的数据分析报告/商业分析报告数据集结合、但数据体量要大得多、并且语料也没有那么干净。在构建大型模型的训练预料时，我们有如下三个基本准则——

1. **规模**：构建大型模型的训练语料需要以T级别数据为目标，这意味着数据集的规模必须能够覆盖大模型所需的丰富知识和复杂模式。幸运的是，无论是在中文社区、还是英文社区中，现在都存在许多规模巨大的语料包供我们选择。在训练10B以下模型时，公开语料本身足以覆盖我们的需求。当然，我们要追求的是规模和质量的平衡、单纯增大规模并无意义。

2. **多样性**：不仅包括语料来源的多样性，还应涵盖覆盖领域、语言的多样性以及文本类型的多样性。语料来源应囊括学术文章、新闻报道、社交媒体内容、技术文档等多种形式，以保证模型适应不同的语境。覆盖领域需要广泛，包括科学、技术、艺术、教育、医学和法律等，以确保模型在各个行业都有所表现。同时，语言的多样性要求语料覆盖常见语言和低资源语言，满足多语言模型的训练需求。文本类型的多样性同样重要，应包含叙述性文本、对话文本、代码样本等，从而增强模型在不同应用场景中的表现。

3. **质量**：高质量的数据是训练优秀模型的前提。语料需要经过严格的清洗与筛选，确保其语法正确、内容无噪声且无重复。同时，数据应具备良好的可读性，避免低质量或不完整的文本进入数据集。此外，语料的合法性和道德合规性至关重要，需要确保数据来源符合法律要求，不包含敏感信息或侵犯隐私的内容。高质量的标注和丰富的元信息也能帮助模型更好地理解数据，从而提升训练效果。

在MateConv构建过程中，我们使用了55%中文、37%英文及其他语言、8%代码的方式进行构建，总数据集大小为4.33TB（当然由于每个数据集的存储格式不同、因此数据集所占用的内存大小本身并不完全代表数据量的大小，但依照存储的格式来判断，通常来说数据量是 csv < Parquet < JSON < JSONL）。为了降低训练成本，我们选择了在数据集打包时就已经经过一定清洗的数据、而没有选择完全没清洗过的数据。当然、这些数据还要再经过清洗流程才能够使用、但耗费的时间与人力会远远低于直接在raw data上进行清洗。

| 数据集编号 | 数据集名称                 | 数据属性（中文/英文/代码）                 | 数据量级           | 存储格式            | 是否经过数据清洗 |
|---|----------------------------|-------------------------------------------|--------------------|---------------------|------------------|
| 1| Skywork-SkyPile 150B       | 中文文本                                      | 620GB             | JSONL 短文本        | 是               |
| 2| wanjuan1.0-nlp-CN             | 中文文本                                 | 580GB | JSONL 短文本压缩  | 是               |
| 3| WuDaoCorporaText | 中文文本                                      | 200GB  | JSONL 短文本        | 是               |
| 4| chinese-fineweb-edu-v2 | 75%中文文本，25%英文文本                                      | 670GB  | Parquet        |   是            |
| 5| Wikipedia-CN | 中文文本                                      | 1.1GB  | JSON        |   是            |
| 6| BaiduBaike-5.63M | 中文文本                                      | 17GB  | JSON        |   是            |
| 7| wangrui6/Zhihu-KOL | 中文问答对                                      | 1.5 GB  | parquet        |   是            |
| 8| wanjuan1.0-nlp-EN             | 英文文本                                 | 440GB | JSONL 短文本压缩  | 是               |
| 9| SlimPajama-627B            | 英文为主的混合语言文本                                      | 1TB               | JSONL 短文本压缩    | 是               |
| 10| Starcoder                  | 代码             | 100+GB/768G             | Parquet            | 是               |
| 11| TheStackDedup                 | 代码             | 700+GB/3TB             | Parquet            | 是               |

在这些数据中、大部分为纯粹的文本数据、有很少的一部分为问答对数据。通常来说，我们不会在预训练阶段加入问答对，但业内有研究声称在预训练阶段加入问答对可以提升模型表现。出于教学目的（教导大家如何处理问答对）以及验证目的（观察是否在预训练阶段加入问答对可以提升模型表现），我们保留了少许问答对数据。同时，除了大型的数据之外，我们还使用了10G上下、1G上下的不同大小的文字数据，如果你没有足够的硬件设备可以拉取这些数据，那你可以拉取小型的数据来体验一下全流程。

- 预训练数据的具体详情

1. **Skywork-SkyPile 150B**

Huggingface 开源，包含 150B 中文 tokens，数据集总大小约 620GB，**采用 JSONL 短文本格式储存**。该数据集从公开可访问的中文互联网网页中获取，经过严格的过滤、去重和敏感数据筛除，以确保数据质量。Skywork-SkyPile数据集被开发者描述为目前中文语料社区中最大的数据集、同时也是我们在众多中文预训练任务中都会看到的一个重要数据集，**在本次训练中我们拉取了Skywork-SkyPile的所有数据来使用**。<br><br>
开源网址：[https://huggingface.co/datasets/Skywork/SkyPile-150B/tree/main/data](https://huggingface.co/datasets/Skywork/SkyPile-150B/tree/main/data)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/14.png)

2. & 8. **wanjuan1.0-nlp**

OpenDataLab开源（**需注册OpenDataLab官网、登录并添加密钥后可访问**）、包含文本数据集、图文数据集、视频数据集三部分，数据总量超过2TB，且已经经过了细粒度的清晰、去重、价值对齐、数据质量较高。其中NLP文本数据集部分包含580G中文数据与440G英文数据，**存储格式为 JSONL 短文本压缩文件格式。在本次训练中我们使用了NLP文本数据集下的所有内容**。

访问网址：[https://opendatalab.org.cn/OpenDataLab/WanJuan1_dot_0](https://opendatalab.org.cn/OpenDataLab/WanJuan1_dot_0) 

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/16.png)

3. **悟道文本数据集（WuDaoCorporaText）**

BAAI（智源社区）开源（**注册智源社区后即可访问**）、包含100TB原始网页数据中清晰得到的最终数据集、开源200G大小、**存储格式为 Jsonl 短文本**。悟道文本数据集是由北京智源人工智能研究院（BAAI）构建的大规模、高质量中文语料库，旨在支持大型语言模型的训练研究。该数据集从公开可获取的网络资源中收集，经过严格的筛选、去重和清洗，确保数据的多样性和高质量。**在本次训练中，我们使用了WuDao的全部数据，但需要注意的是，这个数据无法通过bash脚本直接从网站拉取、只能通过桌面端下载后再上传到服务器。**

访问网址：[https://data.baai.ac.cn/details/WuDaoCorporaText](https://data.baai.ac.cn/details/WuDaoCorporaText) 


4. **Chinese FineWeb Edu V2**

Huggingface开源、包含约 1.88 亿条记录，总计约 4200 亿个tokens，共计679G左右、以 Parquet 格式存储。Chinese FineWeb Edu V2 是一个面向教育领域自然语言处理（NLP）任务的开源中文预训练数据集，数据集经过显著的优化和扩展和清洗。数据虽然以是否具有足够教育意义评估、但实际上数据来源涵盖多个领域（如 IndustryCorpus2、CCI3、TeleChat 等）。**在本次训练中，我们使用了FineWeb Edu V2的全部数据。**

访问链接：[https://huggingface.co/datasets/opencsg/chinese-fineweb-edu-v2](https://huggingface.co/datasets/opencsg/chinese-fineweb-edu-v2)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/23.png)

5. **Wikipedia-cn-20230720-filtered**

huggingface开源、包含约 255,000 条记录，总大小约为 1.1 GB，约 85,000 个标记（tokens）、数据结构为JSON，已经过过滤和清洗。Wikipedia-cn-20230720-filtered 是由 Pleisto 发布的开源中文维基百科数据集。

访问网址：[https://huggingface.co/datasets/pleisto/wikipedia-cn-20230720-filtered](https://huggingface.co/datasets/pleisto/wikipedia-cn-20230720-filtered) 

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/20.png)

6. **BaiduBaike-5.63M**

Huggingface开源、包含约 563 万条百度百科的条目，文件大小约为 16.8 GB、**存储为 JSON 格式**。BaiduBaike-5.63M 是由 Hugging Face 用户 xuqinyang 发布的开源中文百科数据集，由于数据集的条目数量和内容丰富，具体的标记（token）数量可能达到数亿级别。**在本次训练中我们使用了该数据集的全部数据**。

访问网址：[https://huggingface.co/datasets/xuqinyang/BaiduBaike-5.63M](https://huggingface.co/datasets/xuqinyang/BaiduBaike-5.63M) 

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/21.png)

7. **Zhihu-KOL**

Huggingface开源，该数据集包含约 101 万条知乎问答对，数据总大小约为 1.5 GB，**存储为 Parquet 格式**。Zhihu-KOL 是由 Hugging Face 用户 wangrui6 发布的开源中文问答数据集。

访问网址：[https://huggingface.co/datasets/wangrui6/Zhihu-KOL](https://huggingface.co/datasets/wangrui6/Zhihu-KOL) 

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/22.png)


9. **SlimPajama-627B**

Huggingface开源（**需提交邮箱地址、以申请访问权限**）、包含6270亿Token，1T左右总数据大小，**存储格式为Jsonl 短文本压缩文件**。SlimPajama-627B 是由 Cerebras 开发的开源数据集，也是TinyLlaMA项目的训练数据集。该数据集通过对 RedPajama 的 1.2 万亿标记（tokens）进行清洗和去重，精简至 6270 亿标记，删除了约 49.6% 的低质量和重复数据。SlimPajama-627B 包含来自多种语料来源的文本，主要以英语为主，经过严格的去重和清洗，以确保数据的高质量和多样性。**在本次训练中，我们使用了SlimPajama全部的数据。**

访问网址：[https://huggingface.co/datasets/cerebras/SlimPajama-627B](https://huggingface.co/datasets/cerebras/SlimPajama-627B) 

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/18.png)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/17.png)

10. **Starcoder**

Huggingface开源（**需提交邮箱地址、以申请访问权限**）、包含超过 783GB 的代码，总计约 2,500 亿个tokens，涵盖 86 种编程语言，**采用parquet格式存储**。该数据集是由 BigCode 项目发布的开源代码数据集，旨在用于训练大型代码语言模型，信息来源包括 GitHub 提交记录、问题讨论、和 Jupyter 笔记本等多种数据源。数据经过严格的去重和清洗，以确保高质量和多样性。**在本次训练中，我们拉取了Python、SQL、R、Matlab、JavaScript、Java、Json、C、Rust、Go、TypeScript、Kotlin、Swift、Julia、markdown、html等16种语言进行训练**。<br><br>

开源网址：[https://huggingface.co/datasets/bigcode/starcoderdata](https://huggingface.co/datasets/bigcode/starcoderdata)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/15.png)

11. **The Stack Dedup**

Huggingface开源（**需提交邮箱地址、以申请访问权限**），3TB大小、覆盖358种编程语言、数据存储方式为Parquet。The Stack Dedup 是由 BigCode 项目发布的开源代码数据集，旨在为代码大型语言模型（Code LLMs）的预训练提供高质量语料、数据来源于具有宽松许可证的开源代码库，经过严格的去重和清洗，以确保数据的高质量和多样性。

访问网址：[https://huggingface.co/datasets/bigcode/the-stack-dedup](https://huggingface.co/datasets/bigcode/the-stack-dedup) 

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/19_.png)

- 未清洗数据

同时，我们曾考虑过使用现在国内已经非常火热的**MNBVC(Massive Never-ending BT Vast Chinese corpus)超大规模中文语料集**，这个数据集是国内爱好者自发收集的、决定制作成最大的中文开源数据集，目前已收集了40T数据，但数据还没有经过校验、质量参差不齐。走这里可以看到当前这个中文数据集的现状：https://github.com/esbatmop/MNBVC

考虑了很久、我们还是没有使用这个不成熟的数据集。在刚收集好、未清洗的巨量文字数据集上进行清洗是一项耗时耗人工的巨大的工作、需要高度自动化的工具和清晰的策略，同时必须结合人工干预以确保高质量结果、这个数据集完成之时、中文预训练模型的质量也将更上一层楼。现在这个数据集的清洗工作将在社区帮助下逐步完成，如果你感兴趣你可以参与。

- 领域数据集

在数据组成中还有许多不成熟的地方，例如我们其实并没有来得及考虑加入一些更专业的领域的数据（金融、医疗、法律、教育、科技等），如果你的目标模型是一个专业领域的模型，那在预训练阶段加入你的专业文本其实对于后续微调也会很有好处！我们尝试在Huggingface以及各类开源社区中收集更专业的数据集、但是得到的大部分领域数据集尺寸都很小，适合微调但并不适合预训练。后续在微调阶段我们会进一步进行介绍。

- 合成数据集

今年以来、合成数据在大模型训练中的应用逐渐普及，尤其是在预训练和微调阶段，在数据集缺乏的领域、或者在对数据集要求特别高的场合、合成数据集展现了其强大的价值、合成数据集训练也成为目前备受关注的领域之一。

**合成数据具有显著的成本优势**，通过使用大模型生成大规模数据，可以显著降低人工标注和数据收集的成本。在稀有场景、多语言任务或特定领域（如医学和法律）中，合成数据能够快速弥补数据空白，增强语料的多样性。此外，合成数据因其高度可控性，可以根据任务需求设计生成策略，用于提高模型在特定场景中的表现。同时，合成数据还能在保护隐私和遵守法规（如 GDPR）方面发挥关键作用，为敏感数据的训练提供替代方案。

然而，依赖合成数据也面临挑战。**合成数据的质量高度依赖生成模型，如果生成模型本身存在偏差或错误，可能导致数据中出现噪声或不准确的内容，从而影响模型的训练效果**。过度依赖合成数据会引发“模式崩塌”问题，即模型的输出缺乏多样性，重复生成相似内容。此外，合成数据可能无法完全反映真实数据分布，容易导致模型在真实应用场景中表现不佳。在伦理层面，合成数据如果模仿真实内容（如代码或文档），可能引发版权和归属争议，尤其是在大规模使用时。

为了更高效地利用合成数据，必须将其与真实数据结合使用，以平衡数据分布，避免模型过拟合到生成模式。对合成数据的质量控制至关重要，可以通过去噪、筛选和验证等手段确保数据的可靠性。此外，在缺乏真实数据的领域，如低资源语言或稀有任务场景，合成数据的作用尤为显著，可以显著提高模型的性能。未来，随着生成技术的发展和质量评估方法的完善，合成数据将更广泛地应用于大模型的训练，但需要开发者在使用中保持审慎，确保其作为真实数据的有益补充而非单一依赖来源。

未来我们会有更多关于合成数据集的内容被讲解、如果你了解更多信息的话，也欢迎让我们知晓！

### 2.2 巨量数据的拉取与存储

在我们使用几个G甚至几十G大小的数据集时，我们可以通过直接下载到本地、或者直接拉取到服务器的方式来调取，但是当数据集变得巨大时、拉取的过程会变得极其漫长且极其不稳定、且每个数据集可能都被原作者或者开源方定义了独特的拉取方式，因此大型数据的拉取是一个相对复杂的问题。在进行预训练之前，我们需要保证我们的所有数据都能够到位。

我们将以巨型中文数据、英文数据、以及代码数据为例、为你讲解多种不同的数据拉取方式、并且在课件中呈现所有数据的拉取流程，你可以选择任意的数据集进行拉取。

| 数据集编号 | 数据集名称                 | 数据属性（中文/英文/代码）                 | 数据量级           | 存储格式            | 是否经过数据清洗 |
|---|----------------------------|-------------------------------------------|--------------------|---------------------|------------------|
| 1| Skywork-SkyPile 150B       | 中文文本                                      | 620GB             | JSONL 短文本        | 是               |
| 2| wanjuan1.0-nlp-CN             | 中文文本                                 | 580GB | JSONL 短文本压缩  | 是               |
| 3| WuDaoCorporaText | 中文文本                                      | 200GB  | JSONL 短文本        | 是               |
| 4| chinese-fineweb-edu-v2 | 75%中文文本，25%英文文本                                      | 670GB  | Parquet        |   是            |
| 5| Wikipedia-CN | 中文文本                                      | 1.1GB  | JSON        |   是            |
| 6| BaiduBaike-5.63M | 中文文本                                      | 17GB  | JSON        |   是            |
| 7| wangrui6/Zhihu-KOL | 中文问答对                                      | 1.5 GB  | parquet        |   是            |
| 8| wanjuan1.0-nlp-EN             | 英文文本                                 | 440GB | JSONL 短文本压缩  | 是               |
| 9| SlimPajama-627B            | 英文为主的混合语言文本                                      | 1TB               | JSONL 短文本压缩    | 是               |
| 10| Starcoder                  | 代码             | 100+GB/768G             | Parquet            | 是               |
| 11| TheStackDedup                 | 代码             | 700+GB/3TB             | Parquet            | 是               |

- **建立拉取数据专用目录**

下面是**命令行代码**，确保你在运行之前已经按照九天老师的视频建立了MateConv虚拟环境，进入了autodl-tmp这个硬盘存储空间，并且建立了MateConv文件夹。当然，你也可以把目录替换成你自己的目录 ↓

```bash
mkdir -p ~/autodl-tmp/MateConv/Data
```

你也可以为某个数据集单独设置目录——

```bash
mkdir -p ~/autodl-tmp/MateConv/Data/SkyPile
```

你可以查看自己的硬盘空间大小、确保自己有给数据准备好足够的空间（在h之后还有一个空格一个点哦）——

```bash
pwd          # 查看当前路径
df -h .      # 查看当前路径所属磁盘的使用情况
```

返回的结果如下 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/25.png)

这说明还有1.1T的空间供我们使用，你在你自己的磁盘上将会看到相应的空间。你可以在上面的表格中查看具体数据的大小，如果是我们没有标注的数据集，你可以通过Huggingface主页面查看具体的大小，许多数据集都会提到总数据大小，如果没有提到，你也可以通过files页面或者其他页面信息来估算数据总量的大小。

- **设置镜像站**

不同于模型、几乎所有的数据都可以从Huggingface镜像站（[https://hf-mirror.com](https://hf-mirror.com)）进行拉取下载，这样可以避开网络问题、而不用将本地信息代理到线上。因此，在下载数据之前、我们首先要通过设置HF_ENDPOINT环境变量来将我们的下载地址切换到镜像站。

```shell
export HF_ENDPOINT=https://hf-mirror.com     #命令行、设置镜像站环境变量

```

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/24.png)

然后我们有两种方式来拉取数据，一种是使用Huggingface提供的命令行工具`huggingface-cli`来拉取巨型数据、另一种则是可以自定义拉取脚本、结合`aria2c`和`git-lfs`这些工具来更智能、更安全地拉取数据。两种方法分别适用于不同的场景——

| 特性                   | 方法 1：huggingface-cli                           | 方法 2：自定义脚本 + aria2c + git-lfs                       |
|------------------------|--------------------------------------------------|-----------------------------------------------|
| **适用场景**           | 中小型数据集、较小的数据集或一次性下载    | 大型数据集，特别是超大规模数据集、下载耗时长且需要高并发的场景|
| **安装要求**           | 需使用 `huggingface-cli`，但是这一般是huggingface自带的工具           | 需要额外安装 `aria2c` 和 `git-lfs`           |
| **下载速度**           | 受限于单线程速度，可能较慢                        | 支持多线程（如 `-x 16`），下载速度显著提升    |
| **断点续传**           | 理论支持，但大部分实践中无法实现、容易导致重复下载        | 稳定支持，断点续传功能可靠                   |
| **并发支持**           | 不支持                                           | 支持多线程并发下载，提升下载效率             |
| **使用复杂度**         | 简单，官方 CLI 工具，命令较直观                  | 较复杂，**需自定义shell脚本并配置参数**               |
| **镜像支持**           | 支持 `HF_ENDPOINT` 环境变量切换镜像               | 同样支持，可与镜像站搭配使用                 |
| **稳定与适配**             | 对 Hugging Face 官方数据集的兼容性最好<br>但是网络中断或大文件下载时，容易失败                  | 对数据集兼容性较好<br>且在网络不稳定或大文件情况下更稳定             |

#### 2.2.1 huggingface-cli拉取SkyPile数据集

如果你没有安装过huggingface-cli，则需要运行下面的命令行代码👇

```bash
pkgx install huggingface-cli                  #安装huggingface-cli
```

然后你就可以开始拉取数据了！下面帮助你一次性下载完整的数据的命令行代码、请在你的下载目录中准备好至少7-800G的存储空间。<font color="red">**注意！在4MB/s的拉取速度下、下载全部SkyPile数据需要45个小时+7~800G内存，请谨慎运行下面的命令行。**

```bash
huggingface-cli download Skywork/SkyPile-150B --repo-type dataset --resume-download --local-dir ~/autodl-tmp/MateConv/Data/SkyPile  --local-dir-use-symlinks False
```

在这段代码中——
- **`huggingface-cli download`**  调用 Hugging Face 提供的命令行工具，执行下载操作。
  
- **`Skywork/SkyPile-150B`**  指定 Hugging Face 平台上的数据集名称，一般Huggingface上的数据集名称由两部分组成，一个是用户或组织的名称（例如Skywork），另一个是具体的数据集名称（SkyPile-150B）。你需要从Huggingface页面复制正确的名字。

- **`--repo-type dataset`**  明确指定要下载的资源类型是 **数据集**（dataset），如果没有此参数，默认会尝试下载模型（model）。

- **`--resume-download`**  启用断点续传功能。如果之前的下载因网络或其他问题中断，可以从中断处继续下载。但是该功能大部分时候不稳定。

- **`--local-dir ~/autodl-tmp/MateConv/Data/SkyPile`**  指定数据下载后在本地存储的目录路径。在当前代码中，我们是数据将下载到先前建立好的 `~/autodl-tmp/MateConv/Data/SkyPile` 目录。

- **`--local-dir-use-symlinks False`**  关闭符号链接（symlinks）。默认情况下，Hugging Face 会尝试使用符号链接以节省磁盘空间。此参数明确要求不使用符号链接，而是直接将数据下载到指定目录。并且，这一代码现在即将被弃用、因此你可以不写`--local-dir-use-symlinks False`这部分内容。如果你磁盘空间不足、则可以继续尝试开启，并无视掉所有会报的警告。

**如果你想测试一下拉取数据是否成功、则不必一次性拉取全部的数据、而是可以只拉取一部分数据**。我们只需要在原本的代码中间加上特定文件的名字即可👇<font color="red">**对于SkyPile数据拉取一条数据大约需要10min，网速快的情况下可以在5mins内拉取完**——

```bash
huggingface-cli download Skywork/SkyPile-150B data/2020-40_zh_head_0000.jsonl --repo-type dataset --resume-download --local-dir ~/autodl-tmp/MateConv/Data/SkyPile  --local-dir-use-symlinks False
```

在这段命令中，我们只需要在原本的数据集名称后面**增加一个空格、并街上我们要下载的单一文件的目录**，就可以下载单一文件了。同样的，如果我们要拉取多个文件，则可以按照**空格 + 要下载的文件目录**的方式排列相应的代码。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/27.png)

数据的目录具体是什么？你可以在Huggingface页面的files页面下找到👇

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/28.png)

找到文件目录后、点击相应的具体要下载的文件👇

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/29.png)

进行拉取时，你将看到这样的进度条👇

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/30.png)

拉取完成之后，你将可以在你设置的线上目录下找到相应的数据👇

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/31.png)

#### 2.2.2 多线程并行拉取SkyPile数据集

使用`huggingface-cli`拉取数据固然简单，但是最大的困境就在于拉取时间太长、极其容易断线、而且网速极其不稳定。因此针对大型数据、我们要使用Huggingface镜像站官方提供的hfd脚本、并结合`aria2c`和`git-lfs`来进行**多线程并行拉取**。

`hfd.sh` 是由 Hugging Face 镜像站开发的一款专用下载工具，专为 Hugging Face 平台的模型和数据集提供稳定、高速的下载支持。它基于成熟的多线程下载工具 Aria2 构建，结合 Hugging Face 的资源特点，实现了多源分块下载和断点续传功能，能够有效解决大文件下载中的中断问题。通过 `hfd.sh`，用户可以灵活选择下载模型或数据集，并支持包括文件筛选（`--include` 和 `--exclude`）、多线程设置（`-x`）和并行任务配置（`-j`）在内的高度自定义参数，适用于高效管理 Hugging Face 平台上的资源下载需求。

在使用hfd时、还需要单独安装支持库aria2c和git-lfs。其中 Aria2 是一个轻量级、多协议的、**支持多线程分块下载、文件分块下载、多元下载、多服务器下载的数据并行下载工具**，Aria2 可以通过命令行运行，适合集成到脚本中进行自动化下载，且其丰富的参数配置允许用户高度自定义，包括限速、连接数、分块大小等。此外，Aria2 的资源占用极低，即使在性能较低的设备上也能顺畅运行，因而成为开发者和自动化任务中常用的下载工具之一。

而 Git LFS（Git Large File Storage）是 Git 的一种扩展工具，用于优化对大文件的版本控制和存储管理。**它通过将大文件（如音视频文件、图像、模型权重等）存储在外部专用的文件存储中，而不是直接保存在 Git 仓库中，从而避免仓库膨胀问题**。Git LFS 用指针文件替代大文件在版本控制系统中的实际存储，当需要使用这些大文件时，Git LFS 会在后台自动下载和管理它们，确保开发者的操作与普通 Git 工作流无缝兼容。其优点在于有效减少 Git 仓库的体积，提升拉取、克隆和推送等操作的效率，是处理数据密集型项目（如机器学习、音视频编辑等）时的重要工具。

这三个工具结合、可以大幅度提升数据和模型的下载速度和稳定性、可以为开发者提供便捷且高效的下载体验。

接下来我们来看一下首次使用hfd脚本的流程——

**下载hfd.sh文件**、并赋予直接修改命令的权限——

```bash
wget https://hf-mirror.com/hfd/hfd.sh    #下载hfd.sh文件到当前的目录
chmod a+x hfd.sh                         #文件hfd.sh被赋予执行权限
```

**安装aria2c**——

```bash
sudo apt update                            #更新软件包索引
sudo apt install aria2                     #安装aria2c
```

**安装完毕后、需要打印版本号以验证成功**——

```bash
aria2c --version                           #打印版本号
```

出版本号则说明安装成功👇

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/34.png)

**安装git-lfs**——

```bash
sudo apt install git-lfs                   #安装git-lfs
```

**安装完毕后、打印版本号以验证成功**——

```bash
git lfs --version                         #打印版本号
```

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/33.png)

现在可以开始使用`hfd脚本` + `aria2c` + `git-lfs`来拉取数据了。<font color="red">**注意！在4MB/s的拉取速度下、下载下面全部SkyPile数据需要6\~7个小时+7~800G内存，请谨慎运行下面的命令行。**

```bash
./hfd.sh Skywork/SkyPile-150B --dataset --tool aria2c -x 10 --local-dir ~/autodl-tmp/MateConv/Data/SkyPile
```

在这段代码中——

- **`./hfd.sh`**：调用本地的 Bash 脚本 `hfd.sh`，用来从 Hugging Face 平台下载指定的资源。`hfd.sh` 是一个辅助工具，封装了 Hugging Face 的下载逻辑，并支持更多高级功能（如断点续传、多线程、多任务下载）。

- **`Skywork/SkyPile-150B`**：指定 Hugging Face 数据集的 **完整名称**，由组织或用户名称（Skywork）和具体数据集名称（SkyPile-150B）组成。

- **`--dataset`**：指明要下载的是一个 **数据集**（dataset），而不是模型（model）。

- **`--tool aria2c`**：指定使用的下载工具，这里是 `aria2c`，相比于 `wget` 或其他工具，`aria2c` 支持分块下载和多线程，能显著提升下载速度。

- **`-x 10`**：表示设置 **线程数** 为 10，线程数决定了单个文件的下载并发数，`aria2c` 会分块并同时下载多个部分。`-x` 的最大值通常受hfd脚本或工具限制，如果超过限制会报错👇例如设置了-x为16、

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/35.png)

**`--local-dir ~/autodl-tmp/MateConv/Data/SkyPile`**：指定下载文件存储的本地路径。在这里，文件将存储在 `~/autodl-tmp/MateConv/Data/SkyPile` 目录中。如果目录不存在，脚本会尝试自动创建。

同样的、**如果你想测试一下拉取数据是否成功、则不必一次性拉取全部的数据、而是可以只拉取一部分数据**。我们只需要在原本的代码中间加上`--include` + 特定文件的名字即可👇<font color="red">**在4MB/s网络下、对于SkyPile数据拉取一条数据大约需要1mins，网速快的情况下可以在30s内拉取完**——

```bash
./hfd.sh Skywork/SkyPile-150B --dataset --include data/2020-40_zh_head_0001.jsonl --tool aria2c -x 10 --local-dir ~/autodl-tmp/MateConv/Data/SkyPile
```

- **`--include data/2020-40_zh_head_0001.jsonl`**：表示只下载数据集中的指定文件路径。在 `Skywork/SkyPile-150B` 数据集中，这个路径为 `data/2020-40_zh_head_0000.jsonl`。该参数允许使用通配符（如 `*.jsonl`）来匹配多个文件。

> **`hfd脚本`**

作为Hugging Face 镜像站开发的专用下载工具、hfd脚本提供了巨大的可以修改的空间、几乎每个板块都有可以修改的流程，我们重点要关注的有——

- <font color="red">**参数初始化Part**
```bash
REPO_ID=$1
shift

# Default values
TOOL="aria2c"
THREADS=4
CONCURRENT=5
HF_ENDPOINT=${HF_ENDPOINT:-"https://huggingface.co"}
INCLUDE_PATTERNS=()
EXCLUDE_PATTERNS=()
REVISION="main"
```

- **解释**：**这段代码位于整个hfd脚本的开头**。
  - `REPO_ID=$1`：读取第一个参数，作为 Hugging Face 的数据集或模型 ID。
  - `shift`：移除第一个参数，便于解析后续参数。
  - **默认参数**：
    - `TOOL="aria2c"`：默认使用 `aria2c` 作为下载工具。
    - `THREADS=4`：每个文件下载的默认线程数为 4。
    - `CONCURRENT=5`：同时并行下载的文件数为 5。
    - `HF_ENDPOINT`：默认使用 Hugging Face 的主域名 `https://huggingface.co`，<font color="green">**但我们之前已经通过修改环境变量将HF_ENDPOINT这个变量改变为Huggingface镜像站了。**</font>
    - `REVISION="main"`：默认下载主分支内容。

---

- <font color="red">**参数验证和解析Part**
```bash
validate_number() {
    [[ "$2" =~ ^[1-9][0-9]*$ && "$2" -le "$3" ]] || { printf "${RED}[Error] $1 must be 1-$3${NC}\n"; exit 1; }
}

while [[ $# -gt 0 ]]; do
    case $1 in
        --include) ... ;;
        --exclude) ... ;;
        --tool) ... ;;
        -x) validate_number "threads (-x)" "$2" 10; THREADS="$2"; shift 2 ;; 
        -j) validate_number "concurrent downloads (-j)" "$2" 10; CONCURRENT="$2"; shift 2 ;; 
        --dataset) DATASET=1; shift ;;
        --local-dir) LOCAL_DIR="$2"; shift 2 ;;
        --revision) REVISION="$2"; shift 2 ;;
        *) display_help ;;
    esac
done
```

- **解释**：这段代码位于整个脚本的前半段、位于参数初始化part的后面。<br><br>
  - `validate_number`：验证参数是否为正整数，且不超过指定最大值。例如：<br><br>
    - `-x`：线程数（最大 10）。将数据分块、分到不同线程上进行下载。<font color="green">**如果你想要修改最大线程数、可以将`-x) validate_number "threads (-x)" "$2" 10; THREADS="$2"; shift 2 ;;`这行中的10进行修改**。</font>
    > - 一般来说、最大线程数会受到Huggingface等公共资源、以及CPU性能的限制，通常来说我们会将线程数设置在4-8之间（设备比较陈旧、或者设备有限就设置为4）、但最大可以设置到16。<br>
    - `-j`：并行下载数（最大 10）。同步下载多个文件。<font color="green">**如果想要修改并行下载数，可以将`-j) validate_number "concurrent downloads (-j)" "$2" 10; CONCURRENT="$2"; shift 2 ;;`这行中的10进行修改**。</font>
    > - 如果单个文件较大、增加线程数会比较明智、如果是下载多个小文件、增加并行下载数才可以更快完成。<br>
  - `case $1`：解析传入参数，支持以下选项：
    - `--include`：指定下载文件的匹配规则。
    - `--exclude`：排除匹配的文件。
    - `--tool`：选择下载工具（`aria2c` 或 `wget`）。
    - `--local-dir`：指定文件存储路径。
    - `--revision`：选择下载的分支版本。
<br><br>

**该如何选择具体的线程？**

| 场景                          | 推荐 `-x`（线程数） | 推荐 `-j`（并行数） |
|-------------------------------|---------------------|---------------------|
| 普通个人电脑（带宽 ≤ 50Mbps）  | 4-6                 | 2-4                 |
| 高性能电脑（带宽 ≥ 100Mbps）   | 8-16                | 5-8                 |
| 服务器或高速网络（带宽 ≥ 1Gbps）| 16-32               | 10-16               |
| 小文件为主                   | 2-4                 | 8-10                |
| 大文件为主                   | 8-16                | 2-5                 |

需要注意的是、`-x`（线程数）和 `-j`（并行下载数）会同时消耗资源，不建议都设置为过高值。如果 `-x=16`，建议 `-j` 限制在 4-5 左右，反之亦然。大多数情况下、`-x 8`，`-j 4` 是合理起点。高性能设备则可以适当提高到 `-x 16`，`-j 10`。
<br><br>
**你可以使用aria2c测试线程的情况**、测试后根据设备和网络的情况进一步优化，找到适合你的配置。

> 单文件测试线程数（`-x`）、下载大文件时测试：
  ```bash
  aria2c -x 4 https://example.com/large-file.zip
  ```
  - 尝试增加 `-x` 的值，如 8、16，观察是否有明显提速。

> 多文件测试并行数（`-j`）、下载多个小文件时测试：
  ```bash
  aria2c -x 4 -j 8 -i file-list.txt
  ```
  - 增加 `-j` 的值，如 10、16，观察下载时间是否减少。
<br><br>

**除此之外、你还可以通过代码来查看你CPU支持的最大线程数——**

```bash
grep -c processor /proc/cpuinfo          #显示现在CPU上的总线程数
```

输出——

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/36.png)

虽然CPU支持的最大线程是128、但是实际上Huggingface mirror镜像站最大只能支持16。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/37.png)

---

- **文件过滤Part**
```bash
INCLUDE_REGEX=$(printf '%s\n' "${INCLUDE_PATTERNS[@]}" | sed 's/\./\\./g; s/\*/.*/g' | paste -sd '|' -)
EXCLUDE_REGEX=$(printf '%s\n' "${EXCLUDE_PATTERNS[@]}" | sed 's/\./\\./g; s/\*/.*/g' | paste -sd '|' -)
```

- **解释**：
  - 将 `--include` 和 `--exclude` 的文件模式转换为正则表达式，用于筛选需要下载的文件。这样可以实现灵活的文件选择，避免下载整个数据集或模型，提高效率。

---

- **aria2c文件下载Part**
```bash
aria2c --console-log-level=error --file-allocation=none -x "$THREADS" -j "$CONCURRENT" -s "$THREADS" -k 1M -c -i "$fileslist_file"
```

- **解释**：
  - 使用 `aria2c` 下载文件：
    - `-x "$THREADS"`：每个文件的下载线程数、和之前代码中设置的一样。
    - `-j "$CONCURRENT"`：并行下载的文件数、和之前代码中设置的一样。
    - `-s "$THREADS"`：分块下载数、和之前代码中设置的一样。
    - `-k 1M`：设置分块大小为 1MB，如果你的带宽较大、使用更大的分块（比如4M或者8M）能够更充分地发挥网络性能，如果你的网络不稳定你可以设置到1MB或者512kb。
    - `-i "$fileslist_file"`：指定文件列表作为输入。

---

这段脚本支持文件筛选、授权访问、多线程并行下载等功能，适合高效下载 Hugging Face 数据集或模型。你可以通过调整 `-x`、`-j` 等参数优化下载速度，也可以通过 `--include` 精确控制下载的文件。

#### 2.2.3 其他Huggingface数据集的拉取

| 数据集编号 | 数据集名称                 | 数据属性（中文/英文/代码）                 | 数据量级           | 存储格式            | 是否经过数据清洗 |
|---|----------------------------|-------------------------------------------|--------------------|---------------------|------------------|
| 1| Skywork-SkyPile 150B       | 中文文本                                      | 620GB             | JSONL 短文本        | 是               |
| 2| wanjuan1.0-nlp-CN             | 中文文本                                 | 580GB | JSONL 短文本压缩  | 是               |
| 3| WuDaoCorporaText | 中文文本                                      | 200GB  | JSONL 短文本        | 是               |
| 4| chinese-fineweb-edu-v2 | 75%中文文本，25%英文文本                                      | 670GB  | Parquet        |   是            |
| 5| Wikipedia-CN | 中文文本                                      | 1.1GB  | JSON        |   是            |
| 6| BaiduBaike-5.63M | 中文文本                                      | 17GB  | JSON        |   是            |
| 7| wangrui6/Zhihu-KOL | 中文问答对                                      | 1.5 GB  | parquet        |   是            |
| 8| wanjuan1.0-nlp-EN             | 英文文本                                 | 440GB | JSONL 短文本压缩  | 是               |
| 9| SlimPajama-627B            | 英文为主的混合语言文本                                      | 1TB               | JSONL 短文本压缩    | 是               |
| 10| Starcoder                  | 代码             | 100+GB/768G             | Parquet            | 是               |
| 11| TheStackDedup                 | 代码             | 700+GB/3TB             | Parquet            | 是               |

剩下Huggingface数据集、包括opencsg/chinese-fineweb-edu-v2、cerebras/SlimPajama-627B、wangrui6/Zhihu-KOL、xuqinyang/BaiduBaike-5.63M以及pleisto/wikipedia-cn-20230720-filtered数据集，都可以通过和SkyPile类似的方式拉取。具体流程如下👇

1. **设置Huggingface镜像站变量、并确保下载好hfd、设置好目录文件**

```shell
export HF_ENDPOINT=https://hf-mirror.com            #命令行、设置镜像站环境变量

wget https://hf-mirror.com/hfd/hfd.sh               #下载hfd.sh文件到当前的目录
chmod a+x hfd.sh                                    #文件hfd.sh被赋予执行权限

sudo apt update                                     #更新软件包索引
sudo apt install aria2                              #安装aria2c

sudo apt install git-lfs                            #安装git-lfs

#设置目录文件
mkdir -p ~/autodl-tmp/MateConv/Data/Fineweb         
mkdir -p ~/autodl-tmp/MateConv/Data/wikicn       
mkdir -p ~/autodl-tmp/MateConv/Data/baidubaike       
mkdir -p ~/autodl-tmp/MateConv/Data/zhihu   
mkdir -p ~/autodl-tmp/MateConv/Data/slimpajama       
```

2. **`hfd脚本` + `aria2c` + `git-lfs`多线程并行拉取所有数据**

- Fineweb

一次性拉取所有数据的命令👇<font color="red">**注意！在4MB/s、10个线程并行拉取情况下、下载全部fineweb数据需要6\~7小时时间+7\~800G内存，请谨慎运行下面的命令行。**

```bash
./hfd.sh opencsg/chinese-fineweb-edu-v2 --dataset --tool aria2c -x 10 --local-dir ~/autodl-tmp/MateConv/Data/Fineweb
```

单独拉取一个文件的命令👇在10个线程并行拉取情况下、要大约1~2min左右的时间——

```bash
./hfd.sh opencsg/chinese-fineweb-edu-v2 --dataset --include data/00000.parquet --tool aria2c -x 10 --local-dir ~/autodl-tmp/MateConv/Data/Fineweb
```

- SlimPajama

一次性拉取所有数据的命令👇<font color="red">**注意！在4MB/s、10个线程并行拉取情况下、下载全部SlimPajama数据需要个8\~9小时+900G以上内存，请谨慎运行下面的命令行。**

```bash
./hfd.sh cerebras/SlimPajama-627B --dataset --tool aria2c -x 10 --local-dir ~/autodl-tmp/MateConv/Data/slimpajama
```

单独拉取一个文件的命令👇zst文件非常小，在10个线程并行拉取情况下、要大约3s左右的时间——

```bash
./hfd.sh cerebras/SlimPajama-627B --dataset --include train/chunk1/example_train_0.jsonl.zst --tool aria2c -x 10 --local-dir ~/autodl-tmp/MateConv/Data/slimpajama
```

- 小型数据群

其中，zhihu数据大约1.5G、wiki数据大约1G、下载时间都很短、百度百科数据大约17G，下载时间会略长一些，但在多线程并行下速度都不是问题。

```bash
#zhihu
./hfd.sh wangrui6/Zhihu-KOL --dataset --tool aria2c -x 10 --local-dir ~/autodl-tmp/MateConv/Data/zhihu

#Baidubaike
./hfd.sh xuqinyang/BaiduBaike-5.63M --dataset --tool aria2c -x 10 --local-dir ~/autodl-tmp/MateConv/Data/baidubaike

#Wiki
./hfd.sh pleisto/wikipedia-cn-20230720-filtered --dataset --tool aria2c -x 10 --local-dir ~/autodl-tmp/MateConv/Data/wikicn

```

下载好之后，可以在相应的目录下找到你的数据👇

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/50.png)

#### 2.2.4 万卷中英文数据集的拉取

万卷数据集来自Openxlab实验室，因此需要走Openxlab实验室的流程进行下载。

**1. 在Opendatalab首页进行注册、获取自己的Access Key/Secret Access Key**

在这里进行手机号注册 → https://opendatalab.org.cn/OpenDataLab/WanJuan1_dot_0

注册完毕后到这里领取自己的AK/SK → https://sso.openxlab.org.cn/usercenter

> step1——

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/38.png)

> step 2——

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/39.png)

2. **安装openxlab、进行openxlab登录**

> step 1——

```bash
  pip install openxlab #安装
  pip install -U openxlab #版本升级
```

> step 2——

```bash
  openxlab login
```

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/40.png)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/42.png)

3. **设置数据下载目录、查看数据集线上详情**

```bash
mkdir -p ~/autodl-tmp/MateConv/Data/wanjuan

openxlab dataset info --dataset-repo OpenDataLab/WanJuan1_dot_0 #数据集信息及文件列表查看
```

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/43.png)

4. **数据下载**

<font color="red">**注意！wanjuan数据下载是自动多线程并行的、然而在平均网速32MB/s情况下、下载中文+英文nlp方向的wanjuan数据需要12小时+1.5T以上内存，请谨慎运行下面的命令行。**

```bash
#get命令下载整个数据集
openxlab dataset get --dataset-repo OpenDataLab/WanJuan1_dot_0 --source-path /raw/nlp --target-path ~/autodl-tmp/MateConv/Data/wanjuan
```

<font color="red">**注意！在平均网速32MB/s情况下、下载1个4G左右的压缩文件大约需要2分钟时间，你依然可以用下面的代码来测试、流程通了之后再执行完整命令。**

```bash
#download命令特定数据集文件下载
openxlab dataset download --dataset-repo OpenDataLab/WanJuan1_dot_0 --source-path /raw/nlp/CN/ChinaNews-cn/part-006853-a894b46e.jsonl.tar.gz --target-path ~/autodl-tmp/MateConv/Data/wanjuan 
```

- **`openxlab dataset download`**：调用 OpenXLab 提供的命令行工具，用于管理和操作 OpenXLab 平台上的数据。**`dataset download`**：表示操作类型为下载数据集文件。

- **`--dataset-repo OpenDataLab/WanJuan1_dot_0`**：**`--dataset-repo`**：指定目标数据集的仓库名称，和huggingface一样，会包括数据集所属的组织或用户名、同时还会包括数据集本身的名称。你可以在这里找到数据集的仓库全名 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/45.png)

- **`--source-path /raw/nlp/CN/ChinaNews-cn/part-006853-a894b46e.jsonl.tar.gz`**：**`--source-path`**：指定要下载的数据集中的具体文件路径，你可以在这里找到相应的路径。你可以指向一个文件夹、也可以指向具体的数据集文件 ↓
       
![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/46.png)

- **`--target-path ~/autodl-tmp/MateConv/Data/wanjuan`**：指定下载后文件的存储位置。

开始下载之后，会有自动打印和监控流程——

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/44.png)

下载完毕后，你将可以在finalshell的目录中查看到文件👇

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/47.png)

下载完成后、需要进行解压、先单独设置解压后的文件目录——

```bash
mkdir -p ~/autodl-tmp/MateConv/Data/wanjuan/clean            #单独设置解压后的文件目录
```

wanjuan官方提供了解压用的py脚本，可以直接上传到服务器进行使用👇也可直接在线上jupyter进行使用。

```python
import glob 
import os
import time
if __name__ == "__main__":
    dir_list = glob.glob("/data1/step0_rawdata/wanjuan/**/*.jsonl.tar.gz", recursive=True)
    target_root_dir = "~/autodl-tmp/MateConv/Data/wanjuan/clean"
    print(dir_list)
    print(len(dir_list))
    for coutner, file_dir in enumerate(dir_list):
        print(file_dir)
        t0 = time.time()
        save_need_name = file_dir.split("/")[-3:]
        tmp_name = save_need_name[-1]
        # 解压后的文件名称
        tmp_name = tmp_name.replace(".jsonl.tar.gz", ".jsonl")
        tmp_name = os.path.join(target_root_dir, tmp_name)
        # 目标名称带着文件夹名
        save_need_name[-1] = save_need_name[-1].split(".")[0]
        target_file_name = "_".join(save_need_name)
        target_file_name += ".jsonl"
        target_file_name = os.path.join(target_root_dir, target_file_name)
        print("old_name:", tmp_name)
        print("new name:",target_file_name)
        if os.path.exists(target_file_name):
            print(f"have: {target_file_name}")
        cmd = f"tar -zxvf {file_dir} -C {target_root_dir}"
        os.system(cmd)
        os.system(f"mv {tmp_name} {target_file_name}")
        print(coutner, time.time()-t0)
    
    en_dir = os.path.join(target_root_dir, "wanjuan_en")
    cn_dir = os.path.join(target_root_dir, "wanjuan_zh")
    os.mkdir(en_dir)
    os.mkdir(cn_dir)
    mv_cmd1 = f"mv {target_root_dir}/EN*.jsonl {en_dir}"
    os.system(mv_cmd1)
    mv_cmd2 = f"mv {target_root_dir}/CN*.jsonl {cn_dir}"
    os.system(mv_cmd2)

```

同时需要注意的是，<font color="green">**由于wanjuan数据集在下载的时候需要登录到Openxlab，因此在开始下载其他数据集之前、可以尝试重开Bash、并需要重新设置其他数据集所需要的下载环境变量、镜像站等信息**。

#### 2.2.5 WuDao数据集的拉取

**WuDao数据集不支持命令行方式下载、因此只能从网页下载并上传到线上服务器**。幸运的是WuDao数据集本身不是很大、只需要关注中间断线情况、耐心下载即可。

1. **进入页面、微信登录注册**：https://data.baai.ac.cn/details/WuDaoCorporaText

2. **网页进行下载、也可以使用迅雷等工具👇**

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/48.png)

3. **上传至服务器👇由于autodl上传有限速，因此推荐和阿里云盘结合使用、可大幅提升上传速度**

具体autodl链接阿里云盘文件参考👉https://www.autodl.com/docs/netdisk/

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/49.png)

#### 2.2.6 代码类数据集的拉取

| 数据集编号 | 数据集名称                 | 数据属性（中文/英文/代码）                 | 数据量级           | 存储格式            | 是否经过数据清洗 |
|---|----------------------------|-------------------------------------------|--------------------|---------------------|------------------|
| 1| Skywork-SkyPile 150B       | 中文文本                                      | 620GB             | JSONL 短文本        | 是               |
| 2| wanjuan1.0-nlp-CN             | 中文文本                                 | 580GB | JSONL 短文本压缩  | 是               |
| 3| WuDaoCorporaText | 中文文本                                      | 200GB  | JSONL 短文本        | 是               |
| 4| chinese-fineweb-edu-v2 | 75%中文文本，25%英文文本                                      | 670GB  | Parquet        |   是            |
| 5| Wikipedia-CN | 中文文本                                      | 1.1GB  | JSON        |   是            |
| 6| BaiduBaike-5.63M | 中文文本                                      | 17GB  | JSON        |   是            |
| 7| wangrui6/Zhihu-KOL | 中文问答对                                      | 1.5 GB  | parquet        |   是            |
| 8| wanjuan1.0-nlp-EN             | 英文文本                                 | 440GB | JSONL 短文本压缩  | 是               |
| 9| SlimPajama-627B            | 英文为主的混合语言文本                                      | 1TB               | JSONL 短文本压缩    | 是               |
| 10| Starcoder                  | 代码             | 100+GB/768G             | Parquet            | 是               |
| 11| TheStackDedup                 | 代码             | 700+GB/3TB             | Parquet            | 是               |

- **权限申请**

代码数据一般都是量非常大的数据、其特点不仅在于数据量大、还需要我们规定特定的编程语言进行下载，因此CLI代码会与传统文字数据集略有差别。首先，代码数据集的下载会需要权限、因此我们要先申请相应的access token——

1. **挂上梯子、注册Huggingface账号**👉https://huggingface.co/

2. **登录后、在个人profile页面找到自己的username**

> - 点击右上角——
![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/51.png)

> - 复制username——
![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/52.png)

3. **登录后、在这个页面下建立属于自己的AccessToken**👉https://huggingface.co/settings/tokens

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/53.png)

4. **在下面的页面中、建立token名字、并且为所需要的数据集申请权限**

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/54_.png)

点击create token、获得token后直接复制，**你将不会有第二次复制token的机会、因此务必要在这个时候复制token**。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/56.png)

token建好后，可以在token页面中看到、但此时你只能看到你的token名称、将不能再复制具体的access token码了。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/57.png)

---

- **jq工具配置**

与文字数据不同的是、代码数据本身的结构比较复杂、元数据的JSON文件可能嵌套了更多层次结构（例如文件分组、标签、语言分类等），因此解析起来会更为缓慢。为此，我们要下载`jq`工具来帮助我们在下载过程中更高效地解析代码数据。

`jq` 是一个专为处理 JSON 数据设计的轻量级命令行工具，能够高效解析、查询、修改和格式化 JSON 文件或数据流。它的语法灵活，支持强大的过滤和数据操作功能，类似于 JSON 格式的 `sed` 或 `awk`。`jq` 的优势在于其速度快、占用资源少，并能以简洁的方式处理复杂的 JSON 结构，而无需编写冗长的代码。安装 `jq` 的主要原因是许多现代工具（如 `hfd.sh`）在处理大型 JSON 文件时可以借助它快速解析元数据，显著提高文件筛选和匹配的效率，从而避免使用较慢的替代方案（如 `grep` 或 `awk`）。对于需要处理 JSON 数据的开发者和运维人员来说，`jq` 是不可或缺的工具之一。

下面是配置`jq`库的命令行代码👇

```bash
sudo apt update                                  #更新软件包索引
sudo apt install jq -y                           #安装jq

jq --version                                     #检查版本号
```

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/58.png)

---

- **数据拉取**

配置好jq工具后，可以开始进行正式的数据下载了！首先还是要设置好相应的目录——

1. **设置Huggingface镜像站变量、并确保下载好hfd、设置好目录文件**

```shell
export HF_ENDPOINT=https://hf-mirror.com            #命令行、设置镜像站环境变量

wget https://hf-mirror.com/hfd/hfd.sh               #下载hfd.sh文件到当前的目录
chmod a+x hfd.sh                                    #文件hfd.sh被赋予执行权限

sudo apt update                                     #更新软件包索引
sudo apt install aria2                              #安装aria2c

sudo apt install git-lfs                            #安装git-lfs

#设置目录文件
mkdir -p ~/autodl-tmp/MateConv/Data/Starcoder   
mkdir -p ~/autodl-tmp/MateConv/Data/TheStackDedup           
```

2. **`hfd脚本` + `aria2c` + `git-lfs`多线程并行拉取数据**

- **StarCoder**

一次性拉取所有数据的命令👇<font color="green">**注意！你需要将下面的hf_token修改为你刚才申请的有权限的token、同时将hf_username修改为你自己的huggingface账户的名字**。

<font color="red">**同时，在均32MB/s速度下、下载全部starcoder数据需要6\~7小时时间+800G以上内存，请谨慎运行下面的命令行。**

```bash
./hfd.sh bigcode/starcoderdata --dataset --tool aria2c -x 10 --local-dir ~/autodl-tmp/MateConv/Data/Starcoder --hf_token xxx --hf_username TsaiTsai0929
```

单独拉取一种语言的命令👇<font color="red">**例如Python、大约22G的数据，在34MB/s速度下、需要大约11~12min左右的时间**——

```bash
./hfd.sh bigcode/starcoderdata --dataset --tool aria2c -x 10 --local-dir ~/autodl-tmp/MateConv/Data/Starcoder --include *python --hf_token xxx --hf_username TsaiTsai0929
```

当前的hfd脚本一次只能支持一个目录、但是对代码数据集我们必然会需要不止一种代码。**在本次训练中，我们拉取了Python、SQL、R、Matlab、JavaScript、Java、Json、C、Rust、Go、TypeScript、Kotlin、Swift、Julia、markdown、html等16种语言进行训练**。因此你可以设置这样的CLI代码、一次性运行👇

```bash
./hfd.sh bigcode/starcoderdata --dataset --tool aria2c -x 10 --local-dir ~/autodl-tmp/MateConv/Data/Starcoder --include *sql* --hf_token xxx --hf_username TsaiTsai0929

./hfd.sh bigcode/starcoderdata --dataset --tool aria2c -x 10 --local-dir ~/autodl-tmp/MateConv/Data/Starcoder --include *r* --hf_token xxx --hf_username TsaiTsai0929

./hfd.sh bigcode/starcoderdata --dataset --tool aria2c -x 10 --local-dir ~/autodl-tmp/MateConv/Data/Starcoder --include *matlab* --hf_token xxx --hf_username TsaiTsai0929

```

- **TheStackDedup**

一次性拉取所有数据的命令👇<font color="green">**注意！你需要将下面的hf_token修改为你刚才申请的有权限的token、同时将hf_username修改为你自己的huggingface账户的名字**。

<font color="red">**同时，在均32MB/s速度下、下载全部starcoder数据需要24小时时间+3T以上内存，请谨慎运行下面的命令行。**

```bash
./hfd.sh bigcode/the-stack-dedup --dataset --tool aria2c -x 10 --local-dir ~/autodl-tmp/MateConv/Data/TheStackDedup --hf_token xxx --hf_username TsaiTsai0929
```

单独拉取一种语言的命令👇<font color="red">**例如sql、大约4G的数据，在34MB/s速度下、需要大约3~4min左右的时间**——

```bash
./hfd.sh bigcode/the-stack-dedup --dataset --tool aria2c -x 10 --local-dir ~/autodl-tmp/MateConv/Data/TheStackDedup --include *sql --hf_token xxx --hf_username TsaiTsai0929
```

至此、所有数据都拉取完毕了！下载好所有的数据后、你可能发现了、每个数据的格式、状态、结构都不尽相同，因此**我们还需要针对每一个数据进行特定的预处理**。与下载一样、对这些数据进行预处理的流程大多需要分布式工具来辅助我们，接下来让我们一起看看巨量数据的预处理流程。

| 数据集编号 | 数据集名称                 | 数据属性（中文/英文/代码）                 | 数据量级           | 存储格式            | 是否经过数据清洗 |
|---|----------------------------|-------------------------------------------|--------------------|---------------------|------------------|
| 1| Skywork-SkyPile 150B       | 中文文本                                      | 620GB             | JSONL 短文本        | 是               |
| 2| wanjuan1.0-nlp-CN             | 中文文本                                 | 580GB | JSONL 短文本压缩  | 是               |
| 3| WuDaoCorporaText | 中文文本                                      | 200GB  | JSONL 短文本        | 是               |
| 4| chinese-fineweb-edu-v2 | 75%中文文本，25%英文文本                                      | 670GB  | Parquet        |   是            |
| 5| Wikipedia-CN | 中文文本                                      | 1.1GB  | JSON        |   是            |
| 6| BaiduBaike-5.63M | 中文文本                                      | 17GB  | JSON        |   是            |
| 7| wangrui6/Zhihu-KOL | 中文问答对                                      | 1.5 GB  | parquet        |   是            |
| 8| wanjuan1.0-nlp-EN             | 英文文本                                 | 440GB | JSONL 短文本压缩  | 是               |
| 9| SlimPajama-627B            | 英文为主的混合语言文本                                      | 1TB               | JSONL 短文本压缩    | 是               |
| 10| Starcoder                  | 代码             | 100+GB/768G             | Parquet            | 是               |
| 11| TheStackDedup                 | 代码             | 700+GB/3TB             | Parquet            | 是               |

### 2.3 巨量数据清洗与数据预处理